In [ ]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-


import re
from datetime import datetime
import time
import os
import tensorflow as tf
import csv
import pandas as pd
from pandas import Series, DataFrame
from numpy import array
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,2,3,4"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True



INPUT_NODE = 2304
OUTPUT_NODE = 7
LAYER1_NODE = 4096
LAYER2_NODE = 512
LAYER3_NODE = 64
BATCH_SIZE = 1024
LEARNING_RATE_BASE = 0.01
LEARNING_RATE_DECAY = 0.992
REGULARIZER = 0.0001
STEPS = 100000
MOVING_AVERAGE_DECAY = 0.99
TEST_INTERVAL_SECS = 11
MODEL_SAVE_PATH="check_point_next"
MODEL_NAME="data_model"



def get_weight(shape, regularizer):
    w = tf.Variable(tf.truncated_normal(shape, stddev=0.1,mean=0))
    #损失函数loss含正则化regularization
    if regularizer != None: tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(regularizer)(w))
    return w


def get_bias(shape):
    b = tf.Variable(tf.zeros(shape))
    return b


def conv2d(x, W):
    # stride[1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] =1，意思是不对样本个数和channel进行卷积
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")  # padding="SAME"用零填充边界


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")


def forward(x, regularizer):
 
    x = tf.reshape(x, [-1, 48, 48, 1])
    
    ## convl layer ##
    W_conv1 = get_weight([5,5,1,128],None) # kernel 5*5, channel is 1, out size 32
    b_conv1 = get_bias([128])
    h_conv1 = tf.nn.relu(conv2d(x,W_conv1) + b_conv1)  # output size 28*28*32
    h_pool1 = max_pool_2x2(h_conv1)                          # output size 14*14*32
    
    ## conv2 layer ##
    W_conv2 = get_weight([5,5,128,64],None) # kernel 5*5, in size 32, out size 64
    b_conv2 = get_bias([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)  # output size 14*14*64
    h_pool2 = max_pool_2x2(h_conv2)                          # output size 7*7*64
    
    ## conv3 layer ##
    W_conv3 = get_weight([5,5,64,32],None) # kernel 5*5, in size 32, out size 64
    b_conv3 = get_bias([32])
    h_conv3 = tf.nn.relu(conv2d(h_pool2,W_conv3) + b_conv3)  # output size 14*14*64
    h_pool3 = max_pool_2x2(h_conv3)                          # output size 7*7*64
    
    ## funcl layer ##
    W_fc1 = get_weight([1*6*6*32, 576], regularizer)
    b_fc1 = get_bias([576])
    
    h_pool2_flat = tf.reshape(h_pool3, [-1, 1*6*6*32])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob = 0.5)
    
    ## func2 layer ##
    W_fc2 = get_weight([576, 128], regularizer)
    b_fc2 = get_bias([128])
    y1 = tf.nn.relu(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)
    
    W_fc3 = get_weight([128, OUTPUT_NODE], regularizer)
    b_fc3 = get_bias([OUTPUT_NODE])
    y = (tf.matmul(y1,W_fc3)+b_fc3)
    tf.add_to_collection('pred_network', y)  # 用于加载模型获取要预测的网络结构
    return W_fc1,b_fc1,h_fc1,W_fc2,b_fc2,y



def test(new_data):
    
    np.random.shuffle(new_data)
    new_data = new_data[0:1500]
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32, [None, INPUT_NODE])
        y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE])
        keep_prob = tf.placeholder(tf.float32)
        w1, b1, y1, w2, b2, y = forward(x, None)

        xs = new_data[:, :2304]

        ys = new_data[:, 2304:]

        ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        ema_restore = ema.variables_to_restore()
        saver = tf.train.Saver(ema_restore)

        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        while True:
            with tf.Session(config=config) as sess:
                ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score = sess.run(accuracy, feed_dict={x: xs, y_: ys, keep_prob : 0.5})
                    print("After %s training step(s), test accuracy = %g" % (global_step, accuracy_score))
                else:
                    print('No checkpoint file found')
                    return
            time.sleep(TEST_INTERVAL_SECS)




def main():
    
    train=False
    
    if train == True: 
        print("train!")
        data = pd.read_csv("fer2013.csv",low_memory=False).values
        data = data.astype(np.float32)  # change to numpy array and float32
        data = data / 255.0
        
        print("finish fer2013!")
        
        print("数组元素总数：",data.size)      #打印数组尺寸，即数组元素总数  
        print("数组形状：",data.shape)         #打印数组形状 
        print("数组的维度数目",data.ndim)      #打印数组的维度数目
        
        backward(data)
        
    else:
        print("test!")
        data = pd.read_csv("test.csv",low_memory=False).values
        data = data.astype(np.float32)  # change to numpy array and float32
        data = data / 255.0
        print("finish test!")

        print("数组元素总数：",data.size)      #打印数组尺寸，即数组元素总数  
        print("数组形状：",data.shape)         #打印数组形状 
        print("数组的维度数目",data.ndim)      #打印数组的维度数目
        
        test(data)

if __name__ == '__main__':

    main()

    


test!
finish test!
数组元素总数： 16586047
数组形状： (7177, 2311)
数组的维度数目 2
INFO:tensorflow:Restoring parameters from check_point_next/data_model-1
After 1 training step(s), test accuracy = 0.12
INFO:tensorflow:Restoring parameters from check_point_next/data_model-1
After 1 training step(s), test accuracy = 0.12
INFO:tensorflow:Restoring parameters from check_point_next/data_model-1
After 1 training step(s), test accuracy = 0.12
INFO:tensorflow:Restoring parameters from check_point_next/data_model-1
After 1 training step(s), test accuracy = 0.126667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-2
After 2 training step(s), test accuracy = 0.192
INFO:tensorflow:Restoring parameters from check_point_next/data_model-2
After 2 training step(s), test accuracy = 0.202667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-99
After 99 training step(s), test accuracy = 0.234
INFO:tensorflow:Restoring parameters from check_point_next/data_model-196
After 196 traini

After 3386 training step(s), test accuracy = 0.47
INFO:tensorflow:Restoring parameters from check_point_next/data_model-3482
After 3482 training step(s), test accuracy = 0.456
INFO:tensorflow:Restoring parameters from check_point_next/data_model-3579
After 3579 training step(s), test accuracy = 0.460667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-3579
After 3579 training step(s), test accuracy = 0.452
INFO:tensorflow:Restoring parameters from check_point_next/data_model-3676
After 3676 training step(s), test accuracy = 0.481333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-3676
After 3676 training step(s), test accuracy = 0.48
INFO:tensorflow:Restoring parameters from check_point_next/data_model-3772
After 3772 training step(s), test accuracy = 0.463333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-3869
After 3869 training step(s), test accuracy = 0.472
INFO:tensorflow:Restoring parameters from check_point_next/d

INFO:tensorflow:Restoring parameters from check_point_next/data_model-7156
After 7156 training step(s), test accuracy = 0.482
INFO:tensorflow:Restoring parameters from check_point_next/data_model-7252
After 7252 training step(s), test accuracy = 0.492667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-7252
After 7252 training step(s), test accuracy = 0.484
INFO:tensorflow:Restoring parameters from check_point_next/data_model-7349
After 7349 training step(s), test accuracy = 0.486667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-7349
After 7349 training step(s), test accuracy = 0.494
INFO:tensorflow:Restoring parameters from check_point_next/data_model-7446
After 7446 training step(s), test accuracy = 0.481333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-7542
After 7542 training step(s), test accuracy = 0.490667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-7542
After 7542 training step(s), te

INFO:tensorflow:Restoring parameters from check_point_next/data_model-10829
After 10829 training step(s), test accuracy = 0.49
INFO:tensorflow:Restoring parameters from check_point_next/data_model-10926
After 10926 training step(s), test accuracy = 0.488
INFO:tensorflow:Restoring parameters from check_point_next/data_model-10926
After 10926 training step(s), test accuracy = 0.484667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-11022
After 11022 training step(s), test accuracy = 0.512
INFO:tensorflow:Restoring parameters from check_point_next/data_model-11022
After 11022 training step(s), test accuracy = 0.479333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-11119
After 11119 training step(s), test accuracy = 0.494
INFO:tensorflow:Restoring parameters from check_point_next/data_model-11119
After 11119 training step(s), test accuracy = 0.482
INFO:tensorflow:Restoring parameters from check_point_next/data_model-11216
After 11216 training st

After 14406 training step(s), test accuracy = 0.491333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-14502
After 14502 training step(s), test accuracy = 0.480667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-14502
After 14502 training step(s), test accuracy = 0.494
INFO:tensorflow:Restoring parameters from check_point_next/data_model-14599
After 14599 training step(s), test accuracy = 0.489333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-14599
After 14599 training step(s), test accuracy = 0.489333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-14696
After 14696 training step(s), test accuracy = 0.494
INFO:tensorflow:Restoring parameters from check_point_next/data_model-14792
After 14792 training step(s), test accuracy = 0.496667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-14792
After 14792 training step(s), test accuracy = 0.48
INFO:tensorflow:Restoring parameters f

...

After 47079 training step(s), test accuracy = 0.496667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-47079
After 47079 training step(s), test accuracy = 0.498667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-47176
After 47176 training step(s), test accuracy = 0.493333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-47272
After 47272 training step(s), test accuracy = 0.498667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-47272
After 47272 training step(s), test accuracy = 0.498
INFO:tensorflow:Restoring parameters from check_point_next/data_model-47369
After 47369 training step(s), test accuracy = 0.488
INFO:tensorflow:Restoring parameters from check_point_next/data_model-47369
After 47369 training step(s), test accuracy = 0.484667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-47466
After 47466 training step(s), test accuracy = 0.508
INFO:tensorflow:Restoring parameters 

After 50752 training step(s), test accuracy = 0.478667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-50752
After 50752 training step(s), test accuracy = 0.488667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-50849
After 50849 training step(s), test accuracy = 0.5
INFO:tensorflow:Restoring parameters from check_point_next/data_model-50946
After 50946 training step(s), test accuracy = 0.488667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-50946
After 50946 training step(s), test accuracy = 0.487333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-51042
After 51042 training step(s), test accuracy = 0.494
INFO:tensorflow:Restoring parameters from check_point_next/data_model-51139
After 51139 training step(s), test accuracy = 0.493333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-51139
After 51139 training step(s), test accuracy = 0.492
INFO:tensorflow:Restoring parameters fr

INFO:tensorflow:Restoring parameters from check_point_next/data_model-54716
After 54716 training step(s), test accuracy = 0.491333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-54812
After 54812 training step(s), test accuracy = 0.486667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-54812
After 54812 training step(s), test accuracy = 0.487333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-54909
After 54909 training step(s), test accuracy = 0.488667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-55006
After 55006 training step(s), test accuracy = 0.494667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-55006
After 55006 training step(s), test accuracy = 0.504667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-55102
After 55102 training step(s), test accuracy = 0.481333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-55199
After 5

After 58679 training step(s), test accuracy = 0.479333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-58776
After 58776 training step(s), test accuracy = 0.490667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-58776
After 58776 training step(s), test accuracy = 0.502
INFO:tensorflow:Restoring parameters from check_point_next/data_model-58872
After 58872 training step(s), test accuracy = 0.5
INFO:tensorflow:Restoring parameters from check_point_next/data_model-58969
After 58969 training step(s), test accuracy = 0.485333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-58969
After 58969 training step(s), test accuracy = 0.487333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-59066
After 59066 training step(s), test accuracy = 0.489333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-59066
After 59066 training step(s), test accuracy = 0.494667
INFO:tensorflow:Restoring parameters

INFO:tensorflow:Restoring parameters from check_point_next/data_model-62642
After 62642 training step(s), test accuracy = 0.494667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-62739
After 62739 training step(s), test accuracy = 0.503333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-62836
After 62836 training step(s), test accuracy = 0.486667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-62836
After 62836 training step(s), test accuracy = 0.489333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-62932
After 62932 training step(s), test accuracy = 0.486667
INFO:tensorflow:Restoring parameters from check_point_next/data_model-63029
After 63029 training step(s), test accuracy = 0.5
INFO:tensorflow:Restoring parameters from check_point_next/data_model-63029
After 63029 training step(s), test accuracy = 0.477333
INFO:tensorflow:Restoring parameters from check_point_next/data_model-63126
After 63126 